In [51]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy import linalg
import seaborn as sns

#reference: https://github.com/yishantao/Black-Litterman/blob/master/black_litterman.py
def blacklitterman(returns, tau, P, Q):
    """计算加入主观观点后的后验分布之期望值、协方差"""
    mu = returns.mean()  #
    sigma = returns.cov()
    pi1 = mu
    ts = tau * sigma
    Omega = np.dot(np.dot(P, ts), P.T) * np.eye(Q.shape[0])
    middle = linalg.inv(np.dot(np.dot(P, ts), P.T) + Omega)
    er = np.expand_dims(pi1, axis=0).T + np.dot(np.dot(np.dot(ts, P.T), middle),
                                                (Q - np.expand_dims(np.dot(P, pi1.T), axis=1)))
    posteriorSigma = sigma + ts - np.dot(ts.dot(P.T).dot(middle).dot(P), ts)
    return [er, posteriorSigma]


def blminVar(posteriorSigma):
    num_assets = len(posteriorSigma)

    def portfolio_variance(w):
        w = np.array(w)
        vola_w = np.sqrt(np.dot(w.T, np.dot(posteriorSigma * 252, w)))
        return vola_w

    initial_guess = np.ones(num_assets) / num_assets

    # 约束条件（资产权重和为1）
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

    bounds = tuple((0, 1) for asset in range(num_assets))

    result = minimize(portfolio_variance, initial_guess,
                      method='SLSQP', bounds=bounds, constraints=constraints)

    return result.x


In [52]:
ret = pd.read_csv("ret.csv",index_col = 0)
ret.head()

,平安银行,贵州茅台,万科A,工商银行,中国石油
trade_date,,,,,
2015-01-06,-0.014981,-0.023158,-0.036888,0.007905,-0.016821
2015-01-07,-0.019011,-0.024718,-0.009053,-0.011765,0.059880
2015-01-08,-0.033592,-0.006116,-0.044975,-0.041667,-0.014528
2015-01-09,0.008021,-0.007562,-0.010302,0.010352,-0.011466
2015-01-12,-0.020557,-0.020388,-0.024535,-0.012295,-0.037283


In [53]:
ret.describe()

,平安银行,贵州茅台,万科A,工商银行,中国石油
count,2005.000000,2005.000000,2005.000000,2005.000000,2005.000000
mean,0.000116,0.001301,0.000308,0.000026,-0.000199
std,0.021779,0.020302,0.024478,0.013306,0.017226
min,-0.179119,-0.100000,-0.100045,-0.099029,-0.099895
25%,-0.010745,-0.009491,-0.011972,-0.005639,-0.007194
50%,-0.000581,0.000148,0.000000,0.000000,0.000000
75%,0.009790,0.012081,0.009619,0.005300,0.006502
max,0.100351,0.100000,0.100199,0.100000,0.100081


In [54]:
tau = 0.05
"""
平安银行将在未来表现好于其他股票
贵州茅台由于其稳定的增长和市场地位，预计将继续表现出色；但考虑它已经很稳定了，其表现会略低于平安银行
万科A和工商银行预计跟随市场平均水平
中国石油根据平均收益率-0.000199，可能会低于市场平均水平
"""
P = np.array(
    [[1,0,0,0,0],
     [0,1,0,0,0],
     [0,0,0,0,-1]
    ]
)
"""平安银行的预期超额回报为5%（0.05）
贵州茅台的预期超额回报为3%（0.03）
中国石油的预期超额回报为-2%（-0.02）"""
Q = np.array([0.05, 0.03, -0.02]).reshape(-1, 1)

posteriorSigma = blacklitterman(ret, tau, P, Q)
print("后验均值:\n", posteriorSigma[0])

# 计算后验预期回报的方差
print("后验预期回报方差:\n", posteriorSigma[1])

# 使用blminVar函数求解最小方差投资组合
optimal_weights = blminVar(posteriorSigma[1])

print("最优资产配置权重:{}%".format(np.round(optimal_weights*100,2)))

后验均值:
 [[0.02865816]
 [0.02060627]
 [0.01763077]
 [0.01128923]
 [0.01429064]]
后验预期回报方差:
           平安银行      贵州茅台       万科A      工商银行      中国石油
平安银行  0.000485  0.000188  0.000262  0.000166  0.000156
贵州茅台  0.000188  0.000422  0.000142  0.000085  0.000082
万科A   0.000262  0.000142  0.000625  0.000130  0.000136
工商银行  0.000166  0.000085  0.000130  0.000184  0.000109
中国石油  0.000156  0.000082  0.000136  0.000109  0.000304
最优资产配置权重:[ 0.   18.38  2.77 56.77 22.08]%
